In [479]:
import pandas as pd
import os,re,sys
import requests
import logging
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.ensemble import RandomForestClassifier
import string, re,os
import pickle,logging
import requests,json
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import hstack
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from nltk.stem.porter import PorterStemmer
from sklearn.utils import resample

# Download the stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vbasavar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [432]:
df=pd.read_csv("sample30.csv")
df.head()

,id,brand,categories,manufacturer,name,reviews_date,reviews_didPurchase,reviews_doRecommend,reviews_rating,reviews_text,reviews_title,reviews_userCity,reviews_userProvince,reviews_username,user_sentiment
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",Universal Music Group / Cash Money,Pink Friday: Roman Reloaded Re-Up (w/dvd),2012-11-30T06:21:45.000Z,NaN,NaN,5,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,joshua,Positive
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor. This review was collected as part...,Good,NaN,NaN,dorothy w,Positive
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",Lundberg,Lundberg Organic Cinnamon Toast Rice Cakes,2017-07-09T00:00:00.000Z,True,NaN,5,Good flavor.,Good,NaN,NaN,dorothy w,Positive
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-01-06T00:00:00.000Z,False,False,1,I read through the reviews on here before look...,Disappointed,NaN,NaN,rebecca,Negative
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",K-Y,K-Y Love Sensuality Pleasure Gel,2016-12-21T00:00:00.000Z,False,False,1,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,walker557,Negative


### Preprocess function to perform below
1 ) Convert the text to lowercase

2 ) Use porter stemmer to stem the word to its base form

3 ) Remove the stopwords using stopwords module of nltk

In [433]:
def preprocess(document):
    #lowercase
    document = str(document).lower()
    stop_words = stopwords.words("english")
    stemmer = PorterStemmer()
    words = word_tokenize(document)
    words = [stemmer.stem(word) for word in words if word not in stop_words and word not in string.punctuation and re.sub("\d.*", "", word)]
    document = " ".join(words)
    return document


### Select only the required columns from the main dataset

In [434]:
reviews=df[['name','reviews_username','reviews_rating','reviews_text','reviews_title','user_sentiment']]

In [436]:
reviews=reviews[~reviews['reviews_username'].isna()]
reviews=reviews[~reviews['reviews_title'].isna()]
reviews=reviews[~reviews['user_sentiment'].isna()]
reviews.isna().sum()


name                0
reviews_username    0
reviews_rating      0
reviews_text        0
reviews_title       0
user_sentiment      0
dtype: int64

### Preprocess the review title and review text

In [475]:
reviews['reviews_text1']=reviews['reviews_text'].apply(preprocess)
reviews['reviews_title1']=reviews['reviews_title'].apply(preprocess)
reviews.head()


,name,reviews_username,reviews_rating,reviews_text,reviews_title,user_sentiment,reviews_text1,reviews_title1
11114,Clorox Disinfecting Wipes Value Pack Scented 1...,cmtangela,5,"I like knowing things are disinfected, and thi...",Great Disinfectant!,Negative,like know thing disinfect job want ensur cut b...,great disinfect
22337,Alex Cross (dvdvideo),bigmama4773,5,Love Tyler Perry movies! Def a different roll ...,A different Tyler Perry,Positive,love tyler perri movi def differ roll,differ tyler perri
25042,Godzilla 3d Includes Digital Copy Ultraviolet ...,snuffy07,5,This movie revitalizes the classic godzilla th...,Best yet,Positive,movi revit classic godzilla grew,best yet
6045,Red (special Edition) (dvdvideo),chigdon,5,I live Bruce Willis and he didn't let me down ...,great movie,Negative,live bruce willi n't let movi,great movi
21082,Tostitos Bite Size Tortilla Chips,teeree,5,They hold up with whatever dip you are serving.,Chips and dip,Negative,hold whatev dip serv,chip dip


In [476]:
reviews.head()

,name,reviews_username,reviews_rating,reviews_text,reviews_title,user_sentiment,reviews_text1,reviews_title1
11114,Clorox Disinfecting Wipes Value Pack Scented 1...,cmtangela,5,"I like knowing things are disinfected, and thi...",Great Disinfectant!,Negative,like know thing disinfect job want ensur cut b...,great disinfect
22337,Alex Cross (dvdvideo),bigmama4773,5,Love Tyler Perry movies! Def a different roll ...,A different Tyler Perry,Positive,love tyler perri movi def differ roll,differ tyler perri
25042,Godzilla 3d Includes Digital Copy Ultraviolet ...,snuffy07,5,This movie revitalizes the classic godzilla th...,Best yet,Positive,movi revit classic godzilla grew,best yet
6045,Red (special Edition) (dvdvideo),chigdon,5,I live Bruce Willis and he didn't let me down ...,great movie,Negative,live bruce willi n't let movi,great movi
21082,Tostitos Bite Size Tortilla Chips,teeree,5,They hold up with whatever dip you are serving.,Chips and dip,Negative,hold whatev dip serv,chip dip


### Address the data imbalance

1 ) Address the data imbalance with respect to user_sentiment

2 ) Shuffle the dataset

In [439]:
def address_imbalance_binary(reviews):
    col='user_sentiment'
    low = reviews[reviews[col] == 'Negative']
    high = reviews[reviews[col] == 'Positive']

    resample_low = resample(low, replace=True, n_samples=len(high), random_state=1)
    output = pd.concat([high, resample_low])
    output=output.sample(frac = 1)
    return output

In [441]:
# Call the data imbalance function to address the issue

print("-------------Before addressing the data imbalance---------------")
print(reviews.user_sentiment.value_counts())
reviews=address_imbalance_binary(reviews)
print("-------------After addressing the data imbalance----------------")
print(reviews.user_sentiment.value_counts())

(52824, 8)
Negative    26412
Positive    26412
Name: user_sentiment, dtype: int64


### Function to create classification model and save the pickle files for later use

In [443]:
def build_predict_save_model():
    reviews.head()
    X_train, X_test, y_train, y_test = train_test_split(reviews[['reviews_text1','reviews_title1']], reviews['user_sentiment'], random_state=1)
    vect=TfidfVectorizer()
    vect1=TfidfVectorizer()
    
    vect.fit(X_train['reviews_text1'])
    X_train_transformed=vect.fit_transform(X_train['reviews_text1'])
    X_test_transformed=vect.transform(X_test['reviews_text1'])
    
    vect1.fit(X_train['reviews_title1'])
    X_train_cats=vect1.fit_transform(X_train['reviews_title1'])
    X_test_cats=vect1.transform(X_test['reviews_title1'])
    
    x_train_final=hstack((X_train_transformed,X_train_cats))
    x_test_final=hstack((X_test_transformed,X_test_cats))

    rfc = RandomForestClassifier()
    rfc.fit(x_train_final, y_train)
    y_pred_class = rfc.predict(x_test_final)
    accuracy=str(round(metrics.accuracy_score(y_test, y_pred_class) * 100, 2)) + "%"
    print(f"category Model accuracy rfc: {accuracy}")

    
    with open("category_model_allG_rfc.pkl","wb") as f:
        pickle.dump(rfc, f) 

    with open("category_model_allG_vect.pkl", "wb") as f:
        pickle.dump(vect, f) 

    with open("category_model_allG_vect1.pkl", "wb") as f:
        pickle.dump(vect1, f)
        
    mnb = RandomForestClassifier()
    rfc.fit(x_train_final, y_train)
    y_pred_class = rfc.predict(x_test_final)
    accuracy=str(round(metrics.accuracy_score(y_test, y_pred_class) * 100, 2)) + "%"
    print(f"category Model accuracy rfc: {accuracy}")

    
    with open("category_model_allG_mnb.pkl","wb") as f:
        pickle.dump(rfc, f) 

    with open("category_model_allG_vect.pkl", "wb") as f:
        pickle.dump(vect, f) 

    with open("category_model_allG_vect1.pkl", "wb") as f:
        pickle.dump(vect1, f)

In [444]:
build_predict_save_model()

category Model accuracy rfc: 99.17%
category Model accuracy rfc: 99.13%


TfidfVectorizer()

  (0, 2715)	1.0
  (0, 208)	1.0


'Negative'

### Test the model using the pickle files

In [471]:
rfc=pickle.load(open("category_model_allG_rfc.pkl","rb"))
vect=pickle.load(open("category_model_allG_vect.pkl", "rb"))
vect1=pickle.load(open("category_model_allG_vect1.pkl", "rb"))

# Sample the dataset
test=reviews.sample(frac=1).head(20)

In [472]:
for line in test.itertuples():
    review_text=line[5]
    review_body=line[4]
    if rfc.predict(hstack((vect.transform([preprocess(review_body)]),vect1.transform([preprocess(review_text)]))))[0] == "Positive":
        print(line[1])

Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd
Chester's Cheese Flavored Puffcorn Snacks
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total
Clorox Disinfecting Wipes Value Pack Scented 150 Ct Total
L'or233al Paris Elvive Extraordinary Clay Rebalancing Conditioner - 12.6 Fl Oz
Godzilla 3d Includes Digital Copy Ultraviolet 3d/2d Blu-Ray/dvd
Clear Scalp & Hair Therapy Total Care Nourishing Shampoo


In [477]:
nltk

NameError: name 'nltk' is not defined